In [ ]:
# Nativos
import random as rn
import os
import sys
import gc
import json

#calculo
import numpy as np
import pandas as pd
import scipy as sp

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)

from utils import *
#from graphs import *

SEED = 29082013
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

subfolder = "data"
os.listdir(subfolder)

In [ ]:
pageview = pd.read_csv('pageview_trimestral_percent.csv')
print(pageview.shape)

pageview.head()

In [ ]:
pageview.dtypes

In [ ]:
with open('convertions.json', 'r') as outfile_names:
    dict_conversion = json.load(outfile_names)

len(dict_conversion)

In [ ]:
for trim in pageview['trimestre'].unique():
    print(trim, len(dict_conversion[str(trim)]))

In [ ]:
for trim in pageview['trimestre'].unique():
    print(
        trim, 
        len(dict_conversion.get(str(trim + 1), []))
    )

In [ ]:
pageview['hizo_conversion_este_mes'] = pageview[['trimestre', 'USER_ID']].apply(
    lambda _: _[1] in dict_conversion.get(str(_[0]), []), axis=1
).astype(int)

pageview.head()

In [ ]:
pageview['convertion'] = pageview[['trimestre', 'USER_ID']].apply(
    lambda _: _[1] in dict_conversion.get(str(_[0] + 1), []), axis=1
).astype(int)

pageview.head()

In [ ]:
pageview['hizo_conversion_trim_pasado'] = pageview[['trimestre', 'USER_ID']].apply(
    lambda _: _[1] in dict_conversion.get(str(_[0] - 1), []), axis=1
).astype(int)

pageview.head()

In [ ]:
pageview['convertion'] = pageview['convertion'].fillna(0)
pd.crosstab(
    pageview['convertion'], pageview['trimestre']
)

In [ ]:
pageview['hizo_conversion_este_mes'] = pageview['hizo_conversion_este_mes'].fillna(0)
pd.crosstab(
    pageview['hizo_conversion_este_mes'], pageview['trimestre']
)

In [ ]:
pageview['hizo_conversion_trim_pasado'] = pageview['hizo_conversion_trim_pasado'].fillna(0)
pd.crosstab(
    pageview['hizo_conversion_trim_pasado'], pageview['trimestre']
)

In [ ]:
pageview['2_conversiones_seguidas'] = pageview[['hizo_conversion_este_mes', 'hizo_conversion_trim_pasado']].apply(
    lambda _: _[0] == 1 and _[1] == 1, axis=1
).astype(int)

pd.crosstab(
    pageview['2_conversiones_seguidas']  , pageview['trimestre']
)

In [ ]:
pageview.dtypes.head(2)

### SPEED

In [ ]:
spped = pd.read_csv('speed_trimestral_percent.csv')
print(spped.shape)

spped.head()

In [ ]:
print(pageview.shape)
pageview = pageview.merge(spped, how='left', on=["trimestre", "USER_ID"])
print(pageview.shape)

In [ ]:
del spped

### DEVICE

In [ ]:
device = pd.read_csv('device_trimestral_percent.csv')
print(device.shape)

device.head()

In [ ]:
print(pageview.shape)
pageview = pageview.merge(device, how='left', on=["trimestre", "USER_ID"])
print(pageview.shape)

In [ ]:
del device

### SITE_ID

In [ ]:
site_id = pd.read_csv('site_id_trimestral_percent.csv')
print(site_id.shape)

site_id.head()

In [ ]:
print(pageview.shape)
pageview = pageview.merge(site_id, how='left', on=["trimestre", "USER_ID"])
print(pageview.shape)

In [ ]:
del site_id

### SCHEDULE

In [ ]:
schedule = pd.read_csv('schedule_trimestral_percent.csv')
print(schedule.shape)

schedule.head()

In [ ]:
print(pageview.shape)
pageview = pageview.merge(schedule, how='left', on=["trimestre", "USER_ID"])
print(pageview.shape)

In [ ]:
del schedule

### RANGO DÍAS

In [ ]:
dias = pd.read_csv('rango_dias_trimestral_percent.csv')
print(dias.shape)

dias.head()

In [ ]:
print(pageview.shape)
pageview = pageview.merge(dias, how='left', on=["trimestre", "USER_ID"])
print(pageview.shape)

In [ ]:
del dias

### CATEGORIA DE CONTENIDO

In [ ]:
category = pd.read_csv('content_category_trimestral_percent.csv')
print(category.shape)

category.head()

In [ ]:
print(pageview.shape)
pageview = pageview.merge(category, how='left', on=["trimestre", "USER_ID"])
print(pageview.shape)

In [ ]:
del category

### BUSQUEDA

In [ ]:
search = pd.read_csv('search_trimestral_percent.csv')
print(search.shape)

search.head()

In [ ]:
print(pageview.shape)
pageview = pageview.merge(search, how='left', on=["trimestre", "USER_ID"])
print(pageview.shape)

In [ ]:
del search

### PAGINA SOLA

In [ ]:
simple = pd.read_csv('pageview_simple_trimestral_percent.csv')
print(simple.shape)

simple.head()

In [ ]:
print(pageview.shape)
pageview = pageview.merge(simple, how='left', on=["trimestre", "USER_ID"])
print(pageview.shape)

In [ ]:
del simple

In [ ]:
nulos = pageview.isnull().sum()
nulos[nulos > 0]

In [ ]:
pageview = pageview.fillna(0)

In [ ]:
pageview.head()

In [ ]:
historico_conversiones = pageview[['trimestre', 'USER_ID', '2_conversiones_seguidas', 'hizo_conversion_este_mes', 'hizo_conversion_trim_pasado', 'convertion']]
historico_conversiones.head()

In [ ]:
historico_conversiones.shape

In [ ]:
del pageview['2_conversiones_seguidas']
del pageview['hizo_conversion_este_mes']
del pageview['hizo_conversion_trim_pasado']
del pageview['convertion']

In [ ]:
pageview['trimestre'].unique()

In [ ]:
pageview['trimestre'] = pageview['trimestre'].astype(int)

In [ ]:
data = []

for trim in [2, 3, 4]:
    print(trim, trim-1)
    
    grupo = pageview[pageview['trimestre'].isin([trim, trim-1])].groupby(by=['USER_ID']).mean().reset_index()
    grupo['trimestre'] = trim
    data.append(grupo)
    del grupo

In [ ]:
del pageview

In [ ]:
pageview = pd.concat(
    data, 
    ignore_index=True,
    axis=0
)
pageview.head()

In [ ]:
pageview.shape

In [ ]:
pageview.dtypes.head(3)

In [ ]:
historico_conversiones.dtypes.head(3)

In [ ]:
historico_conversiones.groupby(by=['trimestre']).size()

In [ ]:
for trim in historico_conversiones['trimestre'].unique():
    print(trim, historico_conversiones[historico_conversiones['trimestre'] == trim]['USER_ID'].unique().shape)

In [ ]:
pageview.groupby(by=['trimestre']).size()

In [ ]:
for trim in pageview['trimestre'].unique():
    print(trim, 
          pageview[pageview['trimestre'] == trim]['USER_ID'].unique().shape,
          len(set(pageview[pageview['trimestre'] == trim]['USER_ID'].unique()) & set(historico_conversiones[historico_conversiones['trimestre'] == trim]['USER_ID'].unique()))
         )

In [ ]:
#11589 - 11429, 11489 - 11356, 11314 - 11203

In [ ]:
null = historico_conversiones.isnull().sum()
null[null > 0]

In [ ]:
historico_conversiones = historico_conversiones.fillna(0)

In [ ]:
pageview = pageview.merge(
    historico_conversiones, how='left', on=['USER_ID', 'trimestre'])
pageview.head()

In [ ]:
nulo = pageview.isnull().sum()
nulo[nulo > 0]

In [ ]:
pageview = pageview.fillna(0)

# MODELAMIENTO

In [ ]:
from sklearn.linear_model import LassoCV
from sklearn.feature_selection import SelectFromModel
from lightgbm import LGBMClassifier
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, TimeSeriesSplit, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import VarianceThreshold
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier

time_split = TimeSeriesSplit(
    n_splits=10
)

In [ ]:
X_train = pageview[pageview['trimestre'].astype(str).isin(['2', '3'])].drop(['trimestre', 'USER_ID', 'convertion'], axis=1)
X_test = pageview[pageview['trimestre'].astype(str).isin(['4'])].drop(['trimestre', 'USER_ID', 'convertion'], axis=1)
users_test = pageview[pageview['trimestre'].astype(str).isin(['4'])]['USER_ID']
users_train = pageview[pageview['trimestre'].astype(str).isin(['2', '3'])]['USER_ID']
y_train = pageview[pageview['trimestre'].astype(str).isin(['2', '3'])]['convertion']

In [ ]:
users_test.shape

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
#del pageview
gc.collect()

In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold

repeat = RepeatedStratifiedKFold(
    n_splits=4, n_repeats=2, random_state=SEED
)

In [ ]:
lasso_cv = LassoCV(
    cv=repeat, 
    selection='cyclic',
    random_state=SEED, 
    verbose=1,
    normalize=False,
    n_jobs=4
)

sfm = SelectFromModel(lasso_cv)
sfm.fit(
    X_train, y_train
)

In [ ]:
def var_cols_relevants(sfm, X):
    bool_select_var = list(sfm.get_support())
    list_columns = list(X.columns)
    return list({
       list_columns[i]: bool_select_var[i] for i in range(len(list_columns)) if bool_select_var[i]
    }.keys())

In [ ]:
list_var_relevants_r = var_cols_relevants(sfm, X_train)
print(len(list_var_relevants_r))

In [ ]:
X_train[list_var_relevants_r].head()

In [ ]:
lasso_cv = LassoCV(
    cv=time_split, 
    selection='cyclic',
    random_state=SEED, 
    verbose=1,
    normalize=False,
    n_jobs=4
)

sfm = SelectFromModel(lasso_cv)
sfm.fit(
    X_train, y_train
)

In [ ]:
list_var_relevants = var_cols_relevants(sfm, X_train)
print(len(list_var_relevants))

In [ ]:
X_train[list_var_relevants].head()

In [ ]:
print(list_var_relevants)

In [ ]:
print(list_var_relevants_r)

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

best = SelectKBest(chi2, k=300)
best.fit(
    X_train, y_train
)

In [ ]:
list_var_chi = var_cols_relevants(best, X_train)
print(len(list_var_chi))

In [ ]:
X_train[list_var_chi].head()

In [ ]:
print(list_var_chi)

In [ ]:
from sklearn.feature_selection import f_classif

anova = SelectKBest(f_classif, k=300)
anova.fit(
    X_train, y_train
)

In [ ]:
list_var_anova = var_cols_relevants(anova, X_train)
print(len(list_var_anova))

In [ ]:
X_train[list_var_anova].head()

In [ ]:
print(list_var_anova)

In [ ]:
import lightgbm as lgb
fit_params = {
    "eval_metric" : 'auc', 
    'verbose': 2
}

lgbm_grid = {
    'bagging_fraction': [0.65, 0.75, 0.85, 0.95],
    'feature_fraction': [0.15, 0.25, 0.35, 0.45],
    'n_estimators': [150, 450, 750],
    'num_leaves': [33, 65],
    'max_depth': [5, 3, 7],
    'learning_rate': [0.025, 0.05, 0.1, 0.15],
    'min_data_in_leaf': [100, 500],
    'extra_trees': [True, False],
    'deterministic': [True, False],
    'tree_learner': ['serial'],
    'boosting': ['gbdt'],
    'lambda_l2': [0],
    'lambda_l1': [0],
    'is_unbalance': [True],
    'min_gain_to_split': [1]
}

In [ ]:
def apply_lgbm(X_train_part, X_test_part, y_train, users, add='', cv=None):
    model_lgbm = lgb.LGBMClassifier(
        seed=SEED, 
        feature_fraction_seed=SEED, 
        drop_seed=SEED,
        bagging_seed=SEED
    )
    grid = RandomizedSearchCV(
        model_lgbm, lgbm_grid, cv=cv, 
        n_jobs=-1, 
        scoring='roc_auc', 
        verbose=1,
        n_iter=100
    )
    grid.fit(X_train_part, y_train, **fit_params)
    print("-> ", grid.best_score_)
    
    predict_test = grid.best_estimator_.predict_proba(X_test_part)[:,1]
    predict_test = pd.DataFrame(
        {'USER_ID':users, 
         'SCORE':predict_test} 
    )
    display(predict_test.head())
    print(predict_test[predict_test['SCORE'] < 0.01].shape)
    print(predict_test[predict_test['SCORE'] > 0.5].shape)
    print(predict_test[predict_test['SCORE'] > 0.99].shape)
    
    return grid, predict_test

In [ ]:
X_train.shape, X_test.shape

In [ ]:
%%time

grid_1, predict_1 = apply_lgbm(
    X_train[list_var_relevants], 
    X_test[list_var_relevants], 
    y_train, 
    users_test, 
    add='lasso',
    cv=time_split
)
gc.collect()

In [ ]:
%%time

grid_2, predict_2 = apply_lgbm(
    X_train[list_var_chi], 
    X_test[list_var_chi], 
    y_train, 
    users_test, 
    add='chi',
    cv=time_split
)
gc.collect()

In [ ]:
%%time

grid_3, predict_3 = apply_lgbm(
    X_train[list_var_anova], 
    X_test[list_var_anova], 
    y_train, 
    users_test, 
    add='anova',
    cv=time_split
)
gc.collect()

In [ ]:
#grid_3.best_estimator_.feature_importances_

In [ ]:
#print(dir(grid_3.best_estimator_))

### UP

In [ ]:
def _save(predict): 
    sub = pd.read_csv('data/sampleSubmission.zip')
    sub = sub.merge(
        predict, 
        how='left', 
        on=['USER_ID']
    )
    
    sub['SCORE'] = sub['SCORE_y'].combine_first(sub['SCORE_x'])
    del sub['SCORE_y']
    del sub['SCORE_x']
    
    return sub

In [ ]:
predict_1_ = _save(predict_1)
predict_1_.to_csv('lgbm_time_split_lasso_2.csv', index=False)

In [ ]:
predict_2_ = _save(predict_2)
predict_2_.to_csv('lgbm_time_split_chi_2.csv', index=False)

In [ ]:
predict_3_ = _save(predict_3)
predict_3_.to_csv('lgbm_time_split_anova_2.csv', index=False)

### STRATIFICACION

In [ ]:
%%time

grid_1_r, predict_1_r = apply_lgbm(
    X_train[list_var_relevants_r], 
    X_test[list_var_relevants_r], 
    y_train, 
    users_test, 
    add='lasso',
    cv=repeat
)
gc.collect()

In [ ]:
%%time

grid_2_r, predict_2_r = apply_lgbm(
    X_train[list_var_chi], 
    X_test[list_var_chi], 
    y_train, 
    users_test, 
    add='chi',
    cv=repeat
)
gc.collect()

In [ ]:
%%time

grid_3_r, predict_3_r = apply_lgbm(
    X_train[list_var_anova], 
    X_test[list_var_anova], 
    y_train, 
    users_test, 
    add='anova',
    cv=repeat
)
gc.collect()

In [ ]:
predict_1_r_ = _save(predict_1_r)
predict_1_r_.to_csv('lgbm_repeat_stratify_lasso_2.csv', index=False)

In [ ]:
predict_2_r_ = _save(predict_2_r)
predict_2_r_.to_csv('lgbm_repeat_stratify_chi_2.csv', index=False)

In [ ]:
predict_3_r_ = _save(predict_3_r)
predict_3_r_.to_csv('lgbm_repeat_stratify_anova_2.csv', index=False)

In [ ]:
predict_r = predict_1_r_.merge(
    predict_2_r_, how='left', on='USER_ID'
).merge(
    predict_3_r_, how='left', on='USER_ID'
)
predict_r.head()

In [ ]:
predict_r['SCORE'] = predict_r[['SCORE_x', 'SCORE_y', 'SCORE']].mean(axis=1)
predict_r['SCORE'].hist()

In [ ]:
predict_r[['USER_ID','SCORE']].to_csv('lgbm_repeat_stratify_ensamble_2.csv', index=False)

### PREDICT ALL TRIMESTER

In [ ]:
pageview.head()

In [ ]:
def predict_lgbm_all(grid, X_test_part, users, add='', tipo=''):
    predict_test = grid.best_estimator_.predict_proba(X_test_part)[:,1]
    predict_test = pd.DataFrame(
        {'USER_ID':users, 
         'SCORE':predict_test} 
    )
    #display(predict_test.head())
    print(predict_test[predict_test['SCORE'] < 0.01].shape)
    print(predict_test[predict_test['SCORE'] > 0.5].shape)
    print(predict_test[predict_test['SCORE'] > 0.99].shape)
    
    predict_test.groupby(by=['USER_ID'])['SCORE'].mean().to_csv('lgbm_{}_{}_{}_mean.csv'.format(tipo, grid.best_score_, add), index=True)
    predict_test.groupby(by=['USER_ID'])['SCORE'].median().to_csv('lgbm_{}_{}_{}_median.csv'.format(tipo, grid.best_score_, add), index=True)
    return predict_test

In [ ]:
predict_all_1 = predict_lgbm_all(grid_1, pageview[list_var_relevants], pageview['USER_ID'], add='lasso', tipo='time_split')
predict_all_1['SCORE'].hist()

In [ ]:
predict_all_2 = predict_lgbm_all(grid_2, pageview[list_var_chi], pageview['USER_ID'], add='chi', tipo='time_split')
predict_all_2['SCORE'].hist()

In [ ]:
predict_all_3 = predict_lgbm_all(grid_3, pageview[list_var_anova], pageview['USER_ID'], add='anova', tipo='time_split')
predict_all_3['SCORE'].hist()

In [ ]:
predict_all_1_r = predict_lgbm_all(grid_1_r, pageview[list_var_relevants_r], pageview['USER_ID'], add='lasso', tipo='repeat_stratify')
predict_all_1_r['SCORE'].hist()

In [ ]:
predict_all_2_r = predict_lgbm_all(grid_2_r, pageview[list_var_chi], pageview['USER_ID'], add='chi', tipo='repeat_stratify')
predict_all_2_r['SCORE'].hist()

In [ ]:
predict_all_3_r = predict_lgbm_all(grid_3_r, pageview[list_var_anova], pageview['USER_ID'], add='anova', tipo='repeat_stratify')
predict_all_3_r['SCORE'].hist()